In [2]:
import os
import csv
import MDAnalysis
import subprocess
import Bio.PDB
import pandas as pd
import numpy as np
import MDAnalysis.analysis.hbonds
# Naccess for SASA      
# before parse, deal with the pdb files 
path = '/home/tongwade780/test_data_muscle/alinmented_pdb'
outputpath = '/home/tongwade780/test_data_muscle/alinmented_pdb/Naccess_SASA'
files = os.listdir(path)
os.chdir(outputpath)

for onefile in files: 
    if onefile[-3:] == 'pdb': 
        with open(os.path.join(path,onefile),'r') as filetest:
            data = filetest.readlines()
        nonNMR = True
        for line in data:
            if len(line.strip().split()) == 3 and line.strip().split()[-1] == 'NMR':
                NMRfile_MD = MDAnalysis.Universe(os.path.join(path,onefile))
                allatom = NMRfile_MD.select_atoms("all")
                with MDAnalysis.Writer(path+onefile[:-4]+"_1.pdb", allatom.n_atoms) as W:
                    W.write(allatom)
                subprocess.call(["/opt/Naccess/naccess",os.path.join(path,onefile[:-4]+"_1.pdb")])
                nonNMR = False
        if nonNMR and onefile[-6:]!="_1.pdb": 
            subprocess.call(["/opt/Naccess/naccess",os.path.join(path,onefile[:-4]+".pdb")])               

# parse the res files to get the properties of residues 
path ='/home/tongwade780/test_data_muscle/alinmented_pdb/Naccess_SASA'
path_csv = '/home/tongwade780/test_data_muscle/alinmented_pdb/Naccess_SASA/csv'
files = os.listdir(path)
for onefile in files:
    if onefile[-3:] == 'rsa':
        with open(os.path.join(path,onefile), 'r') as naccessData:
            full_data = naccessData.readlines()
            
        with open(os.path.join(path_csv,onefile[:-4]+'_sasa.csv'), 'wb') as output_file:
            output_file_csv = csv.writer(output_file)
            
            for current_line in full_data:
                if current_line[0:3] == 'RES':
                    parsed_line = current_line[10:].strip().split()
                    if current_line[4:7] in ["ALA","CYS","ASP","GLU","PHE","GLY","HIS","ILE","LYS","LEU","MET","ASN","PRO","GLN","ARG","SER","THR","VAL","TRP","TYR"]:
                        output_file_csv.writerow([parsed_line[0], parsed_line[2]])
                        
                        
#pKa
# parse the pka files
propka31Path = '/usr/local/anaconda/bin/propka31'
inputpath = '/home/tongwade780/test_data_muscle/alinmented_pdb'
path = os.path.join(inputpath,'pKa')
path_csv = os.path.join(inputpath,'pKa/csv')
try:
    os.mkdir(path)
except:
    pass
try:
    os.mkdir(path_csv)
except:
    pass
os.chdir(path)
for inputfile in os.listdir(inputpath):
    if inputfile[-3:] == 'pdb':
        command = '{propka31Path} {inputfile}'.format(propka31Path=propka31Path,inputfile=os.path.join(inputpath,inputfile))
        print command
        subprocess.call(command,shell=True)
files = os.listdir(path)
for onefile in files:
    if onefile[-3:] == 'pka':
        with open(os.path.join(path,onefile), 'r') as pKafile:
            full_data = pKafile.readlines()
        
        with open(os.path.join(path_csv,onefile[:-4]+'_pka.csv'), 'wb') as output_file:
            output_file_csv = csv.writer(output_file)
        
            for index, current_line in enumerate(full_data):
                if current_line[0:12] == '       Group':
                    starting_index = index
                    break
        
            for current_line in full_data[starting_index+1:]:
        
                if current_line[0] == '-':
                    break
        
                parsed_line = current_line.strip().split()
        
                if parsed_line[0] == 'C-' or parsed_line[0] == 'N+':
                    continue
                #print [parsed_line[0], parsed_line[1], parsed_line[3], parsed_line[4], str(float(parsed_line[3]) - float(parsed_line[4]))]
                #output_file_csv.writerow([parsed_line[0], parsed_line[1], parsed_line[3], parsed_line[4], str(float(parsed_line[3]) - float(parsed_line[4]))])
                if parsed_line[0] in ["ALA","CYS","ASP","GLU","PHE","GLY","HIS","ILE","LYS","LEU","MET","ASN","PRO","GLN","ARG","SER","THR","VAL","TRP","TYR"]:
                    output_file_csv.writerow([parsed_line[1], parsed_line[3]])

                    
# Dssp (dssp.exe -i 1crn.pdb -o 1crn.dssp)
path = '/home/tongwade780/test_data_muscle/alinmented_pdb'
path_csv = '/home/tongwade780/test_data_muscle/alinmented_pdb/dssp/csv'
#path = '/home/tongwade780/test_20_datas/test_internal_pdb/residTOinternal_pdb/'
#path_csv = '/home/tongwade780/test_20_datas/test_internal_pdb/residTOinternal_pdb/dssp/csv'
dssp_exec ='/opt/dssp/dssp'
files = os.listdir(path)
for onefile in files:
    if onefile[-3:] == 'pdb':
        #print onefile
        idx = pd.IndexSlice
        
        # Use DSSP to assign secondary structure type to each residue.
        pdb_parser = Bio.PDB.PDBParser()
        structure = pdb_parser.get_structure('PDB', os.path.join(path,onefile))
        model = structure[0]
        dssp_results = Bio.PDB.DSSP(model, os.path.join(path,onefile), dssp=dssp_exec)
        
        Uni = MDAnalysis.Universe(os.path.join(path,onefile))
        protein = Uni.select_atoms("protein")
        protein_chain = np.unique(protein.segids)[0]
        
        chain_ID_resid_list = dssp_results.keys()
        dssp_results_table = pd.DataFrame(list(dssp_results))
        dssp_results_table = dssp_results_table[[0,1,2,3,4,5]]
        dssp_results_table.columns = ['DSSP Index', 'Residue Type', 'SS', 'Acc', 'Phi', 'Psi']
        dssp_results_table.index = pd.MultiIndex.from_tuples([(chain_ID_resid_list[x][0], chain_ID_resid_list[x][1][1]) for x in range(len(chain_ID_resid_list))])
        dssp_results_table = dssp_results_table.reindex()
        
        # Make sure all helices have negative phi and psi angles, otherwise change them to '-'
        helix_only_rows = (dssp_results_table['SS'] == 'H')
        helix_only_table = dssp_results_table[helix_only_rows]
        first_chain_helix_only = helix_only_table.loc[protein_chain,:]
        pos_phi_or_psi_idx = first_chain_helix_only[(first_chain_helix_only['Phi'] >= 0).values + (first_chain_helix_only['Psi'] >= 0).values].index.tolist()
        dssp_results_table.loc[idx[protein_chain,pos_phi_or_psi_idx], 'SS'] = '-'
        
        with open(os.path.join(path_csv,onefile[:-4]+'_dssp.csv'), 'wb') as output_file:
            output_file_csv = csv.writer(output_file)
            for i in range(len(dssp_results_table['SS'])):
                if dssp_results_table['SS'][i] == 'H' or dssp_results_table['SS'][i] == 'I' or dssp_results_table['SS'][i] == 'G':
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],0.5])
                elif dssp_results_table['SS'][i] == 'T' or dssp_results_table['SS'][i] == 'S':
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],1])
                elif dssp_results_table['SS'][i] == 'B' or dssp_results_table['SS'][i] == 'E':
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],0])
                else:
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],0])
                            

/usr/local/anaconda/bin/propka31 /home/tongwade780/test_data_muscle/alinmented_pdb/1p5z.pdb
/usr/local/anaconda/bin/propka31 /home/tongwade780/test_data_muscle/alinmented_pdb/1p62.pdb
/usr/local/anaconda/bin/propka31 /home/tongwade780/test_data_muscle/alinmented_pdb/2a7q.pdb
/usr/local/anaconda/bin/propka31 /home/tongwade780/test_data_muscle/alinmented_pdb/1p61.pdb
/usr/local/anaconda/bin/propka31 /home/tongwade780/test_data_muscle/alinmented_pdb/3ipx.pdb
